# Data Warehousing Project
### Group Project
#### CAT Tech | Members: Carmen, Andy and Tenzin

### Project Summary

#### Evaluate the correlation between TSA throughput and the weather June 1st 2023 to September 30th 2023

#### The Transportation Security Administration (TSA) is an agency of the U.S. Department of Homeland Security. It is responsible for ensuring the security of the traveling public in the United States. The TSA's main focus is on aviation security, including screening passengers and their belongings, as well as protecting the nation's transportation systems from potential threats.

#### - I need a map (united states map) that will give me the concentration of passengers.
#### - A line charge that will give the number of passengers by airport per day. Make sure I can select the Airport.


In [ ]:
%%capture
!pip install pdfplumber
!pip install pandas
!pip install azure-storage-blob
!pip install azure.storage.blob


# Extracting the data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import pdfplumber

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Takes about 1hour, advised to run it in a high computing machine, prefferably higer than 16GB
import pdfplumber
import pandas as pd
import os

# Directory containing PDF files
pdf_folder = "/content/drive/MyDrive/CAT/TSA throughput pdfs. "

#Creating an empty dataframe/list to store the data
dataframes = []

# Sorting the pdf files
pdf_files = sorted([pdf_file for pdf_file in os.listdir(pdf_folder) if pdf_file.endswith('.pdf')])

# Iterate over all sorted PDF files in the folder
for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_folder, pdf_file)

    # Opening each PDF file using pdfplumber
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            # Extracting table from the current page
            table = page.extract_table()

            # If a table is found, convert it into a pandas DataFrame and add it to the list
            if table:
                df = pd.DataFrame(table[1:], columns=table[0])
                dataframes.append(df)

# Concatenate all the DataFrames from each PDF
combined_df = pd.concat(dataframes, ignore_index=True)

# Now, 'combined_df' is a DataFrame containing all the data from the PDFs
print(combined_df.head())

# Saving the combined DataFrame to a CSV file
#combined_df.to_csv("A-M_uncleaned.csv", index=False)


        Date Hour\nof\nDay Airport                                 None  \
0  5/28/2023         00:00     ANC  Ted Stevens Anchorage International   
1       None          None     ATL     Hartsfield Atlanta International   
2       None          None     BQN                     Rafael Hernandez   
3       None          None     BTV             Burlington International   
4       None          None     DCA           Washington Reagan National   

               City State            Checkpoint Total\nPax +\nKCM\nPAX  
0         Anchorage    AK      South Checkpoint                    113  
1           Atlanta    GA       Main Checkpoint                    142  
2         Aguadilla    PR  Rafael Hernandez Air                    268  
3  South Burlington    VT       Main Checkpoint                      0  
4         Arlington    VA           Concourse A                      1  


### Uploading to Azure Blob

In [ ]:
# Azure Storage Account information
account_name = "your account name"
account_key = "your account key"
container_name = "jsoncme"
blob_name = 'A-M_cleaned.csv'
local_file_path = 'A-M_cleaned.csv'

# BlobServiceClient
blob_service_client = BlobServiceClient(account_url=f"https://{account_name}.blob.core.windows.net", credential=account_key)

# reference to the container
container_client = blob_service_client.get_container_client(container_name)

# Uploading the file to Azure Blob Storage
with open(local_file_path, "rb") as data:
    container_client.upload_blob(name=blob_name, data=data)

print(f"File '{blob_name}' uploaded to Azure Blob Storage successfully.")

## Loading into our notebook from Azure Blob

In [ ]:
from azure.storage.blob import BlobServiceClient
import pandas as pd
from io import BytesIO

# Azure Storage Account information
account_name = "your account name"
account_key = "your account key"
container_name = "jsoncme"
blob_name = 'A-M_uncleaned.csv'

# BlobServiceClient
blob_service_client = BlobServiceClient(account_url=f"https://{account_name}.blob.core.windows.net", credential=account_key)

# reference to the blob
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

# Downloading the blob content
blob_data = blob_client.download_blob()
content = blob_data.readall()

# Loading the CSV content into a Pandas DataFrame into our notebook
combined_df = pd.read_csv(BytesIO(content))

combined_df



             Date Hour\nof\nDay Airport  \
0       5/28/2023         00:00     ANC   
1             NaN           NaN     ATL   
2             NaN           NaN     BQN   
3             NaN           NaN     BTV   
4             NaN           NaN     DCA   
...           ...           ...     ...   
697151        NaN           NaN     SJC   
697152        NaN           NaN     SJU   
697153        NaN           NaN     SLC   
697154        NaN           NaN     SMF   
697155        NaN           NaN     NaN   

                                    Unnamed: 3              City State  \
0          Ted Stevens Anchorage International         Anchorage    AK   
1             Hartsfield Atlanta International           Atlanta    GA   
2                             Rafael Hernandez         Aguadilla    PR   
3                     Burlington International  South Burlington    VT   
4                   Washington Reagan National         Arlington    VA   
...                                    

## Cleaning the data
### The data since it is converted from a pdf. There are noticeable Nan values that needs to be cleaned.

In [ ]:
# Replacing the blank cells with NaN so that we can front fill it with data
#If we dont fill it wilh Naan and add the datetime, the code front fill doesnt work
# Pandas does automatically fill in Nan but it is better to double check.

combined_df.replace('', pd.NA, inplace=True)

In [ ]:
from datetime import datetime

In [ ]:
combined_df.head(10)

        Date Hour\nof\nDay Airport                           Unnamed: 3  \
0  5/28/2023         00:00     ANC  Ted Stevens Anchorage International   
1        NaN           NaN     ATL     Hartsfield Atlanta International   
2        NaN           NaN     BQN                     Rafael Hernandez   
3        NaN           NaN     BTV             Burlington International   
4        NaN           NaN     DCA           Washington Reagan National   
5        NaN           NaN     NaN                                  NaN   
6        NaN           NaN     DEN                 Denver International   
7        NaN           NaN     NaN                                  NaN   
8        NaN           NaN     DTW           Detroit Metro Wayne County   
9        NaN           NaN     NaN                                  NaN   

               City State            Checkpoint Total\nPax +\nKCM\nPAX  
0         Anchorage    AK      South Checkpoint                    113  
1           Atlanta    GA   

In [ ]:
# Checking for NaN values before front-fill
print("Nan values before front-fill:")
print(combined_df.isna().sum())

# Assigning new column headers
combined_df.columns = ["date", "hour", "airportcode", "airportname", "city", "state", "checkpoint", "total"]

# Removing all rows where 'total' column contains 'Total'
combined_df = combined_df[~combined_df['total'].str.contains('Total', na=False)]

# Front-fill missing data for specific columns
combined_df['date'].fillna(method='ffill', inplace=True)
combined_df['hour'].fillna(method='ffill', inplace=True)
combined_df['airportcode'].fillna(method='ffill', inplace=True)
combined_df['airportname'].fillna(method='ffill', inplace=True)
combined_df['city'].fillna(method='ffill', inplace=True)
combined_df['state'].fillna(method='ffill', inplace=True)
combined_df['total'].fillna(0, inplace=True)

# Reset index and handle missing 'date' and 'hour' with front-fill within a range
combined_df.reset_index(drop=True, inplace=True)
idx_date = combined_df['date'].first_valid_index()
idx_hour = combined_df['hour'].first_valid_index()
combined_df.loc[0:idx_date, 'date'] = combined_df.loc[0:idx_date, 'date'].ffill()
combined_df.loc[0:idx_hour, 'hour'] = combined_df.loc[0:idx_hour, 'hour'].ffill()

# Checking for NaN values after front-fill
print("Nan values after front-fill:")
combined_df.isna().sum()

Nan values before front-fill:
Date                      684198
Hour\nof\nDay             682142
Airport                   286306
Unnamed: 3                286306
City                      286306
State                     286297
Checkpoint                     9
Total\nPax +\nKCM\nPAX         9
dtype: int64
Nan values after front-fill:
date           0
hour           0
airportcode    0
airportname    0
city           0
state          0
checkpoint     9
total          0
dtype: int64


In [ ]:
# displaying the cleaned data
combined_df

,date,hour,airportcode,airportname,city,state,checkpoint,total
0,5/28/2023,00:00,ANC,Ted Stevens Anchorage International,Anchorage,AK,South Checkpoint,113
1,5/28/2023,00:00,ATL,Hartsfield Atlanta International,Atlanta,GA,Main Checkpoint,142
2,5/28/2023,00:00,BQN,Rafael Hernandez,Aguadilla,PR,Rafael Hernandez Air,268
3,5/28/2023,00:00,BTV,Burlington International,South Burlington,VT,Main Checkpoint,0
4,5/28/2023,00:00,DCA,Washington Reagan National,Arlington,VA,Concourse A,1
...,...,...,...,...,...,...,...,...
697151,8/26/2023,23:00,SJC,Norman Y Mineta San Jose International,San Jose,CA,Terminal A,0
697152,8/26/2023,23:00,SJU,Luis Munoz Marin International,San Juan,PR,Checkpoint BC,238
697153,8/26/2023,23:00,SLC,Salt Lake City International,Salt Lake City,UT,Main,84
697154,8/26/2023,23:00,SMF,Sacramento International,Sacramento,CA,Central B,184


### Further analyzing the data, i noticed that there was an false data resulting from the page number in the pdf. See line 53 below under the state.

In [ ]:
combined_df.iloc[50:55]

,date,hour,airportcode,airportname,city,state,checkpoint,total
50,5/28/2023,00:00,PDX,Portland International Airport,Portland,OR,PDX D/E,5
51,5/28/2023,00:00,PHX,Phoenix Sky Harbor International,Phoenix,AZ,T-3 North,6
52,5/28/2023,00:00,PHX,Phoenix Sky Harbor International,Phoenix,AZ,T-4 A,7
53,5/28/2023,00:00,PHX,Phoenix Sky Harbor International,Phoenix,1,NaN,0
54,5/28/2023,00:00,PIT,Pittsburgh International,Pittsburgh,PA,Main Checkpoint,12


In [ ]:
# Thus I removed the entire row with any cell under state that that is an int or a float. I left only the ones in string .
# I dont suppose there is any state in the US with a number unless it is a state founded by Elon Musk
combined_df = combined_df[~combined_df['state'].str.contains('\d', na=False)]

# Reseting the index if needed
combined_df.reset_index(drop=True, inplace=True)


In [ ]:
# Now it is cleaned . I removed the entore row that has the data and moved the following row one step up
# Look for 53, its cleaned
combined_df.iloc[50:55]

,date,hour,airportcode,airportname,city,state,checkpoint,total
50,5/28/2023,00:00,PDX,Portland International Airport,Portland,OR,PDX D/E,5
51,5/28/2023,00:00,PHX,Phoenix Sky Harbor International,Phoenix,AZ,T-3 North,6
52,5/28/2023,00:00,PHX,Phoenix Sky Harbor International,Phoenix,AZ,T-4 A,7
53,5/28/2023,00:00,PIT,Pittsburgh International,Pittsburgh,PA,Main Checkpoint,12
54,5/28/2023,00:00,PSE,Mercedita,Ponce,PR,Mercedita Airport,128


# Converting it date time format
## Why is this crucial is because we need to corelate it with the weather API forecast

## We will corelate using the datetime and the name of the city

In [ ]:
combined_df.head()


,date,hour,airportcode,airportname,city,state,checkpoint,total
0,5/28/2023,00:00,ANC,Ted Stevens Anchorage International,Anchorage,AK,South Checkpoint,113
1,5/28/2023,00:00,ATL,Hartsfield Atlanta International,Atlanta,GA,Main Checkpoint,142
2,5/28/2023,00:00,BQN,Rafael Hernandez,Aguadilla,PR,Rafael Hernandez Air,268
3,5/28/2023,00:00,BTV,Burlington International,South Burlington,VT,Main Checkpoint,0
4,5/28/2023,00:00,DCA,Washington Reagan National,Arlington,VA,Concourse A,1


In [ ]:
# Combine 'date' and 'hour' columns into a new 'datetime' column
combined_df['datetime'] = pd.to_datetime(combined_df['date'] + ' ' + combined_df['hour'])

combined_df


             date   hour airportcode                             airportname  \
0       5/28/2023  00:00         ANC     Ted Stevens Anchorage International   
1       5/28/2023  00:00         ATL        Hartsfield Atlanta International   
2       5/28/2023  00:00         BQN                        Rafael Hernandez   
3       5/28/2023  00:00         BTV                Burlington International   
4       5/28/2023  00:00         DCA              Washington Reagan National   
...           ...    ...         ...                                     ...   
697142  8/26/2023  23:00         SJC  Norman Y Mineta San Jose International   
697143  8/26/2023  23:00         SJU          Luis Munoz Marin International   
697144  8/26/2023  23:00         SLC            Salt Lake City International   
697145  8/26/2023  23:00         SMF                Sacramento International   
697146  8/26/2023  23:00         SMF                Sacramento International   

                    city state         

<ipython-input-26-e3ec1b91ea60>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['datetime'] = pd.to_datetime(combined_df['date'] + ' ' + combined_df['hour'])


In [ ]:

# REDefining the order of columns, this is the order i want.
# IT is better to organise and merge later on with the weather API data
desired_column_order = ['city', 'datetime', 'airportcode', 'airportname', 'state', 'checkpoint', 'total']

# The code that does the rearranging
combined_df = combined_df[desired_column_order]

combined_df

,city,datetime,airportcode,airportname,state,checkpoint,total
0,Anchorage,2023-05-28 00:00:00,ANC,Ted Stevens Anchorage International,AK,South Checkpoint,113
1,Atlanta,2023-05-28 00:00:00,ATL,Hartsfield Atlanta International,GA,Main Checkpoint,142
2,Aguadilla,2023-05-28 00:00:00,BQN,Rafael Hernandez,PR,Rafael Hernandez Air,268
3,South Burlington,2023-05-28 00:00:00,BTV,Burlington International,VT,Main Checkpoint,0
4,Arlington,2023-05-28 00:00:00,DCA,Washington Reagan National,VA,Concourse A,1
...,...,...,...,...,...,...,...
697142,San Jose,2023-08-26 23:00:00,SJC,Norman Y Mineta San Jose International,CA,Terminal A,0
697143,San Juan,2023-08-26 23:00:00,SJU,Luis Munoz Marin International,PR,Checkpoint BC,238
697144,Salt Lake City,2023-08-26 23:00:00,SLC,Salt Lake City International,UT,Main,84
697145,Sacramento,2023-08-26 23:00:00,SMF,Sacramento International,CA,Central B,184


In [ ]:
cities = combined_df['city'].unique()
cities_df = pd.DataFrame({'city': cities})

# Displaying the DataFrame/list with unique cities
print(cities_df)


                 city
0           Anchorage
1             Atlanta
2           Aguadilla
3    South Burlington
4           Arlington
..                ...
398          Gustavus
399         Pago Pago
400            Bishop
401          Loveland
402            Alpena

[403 rows x 1 columns]


In [ ]:
%%capture
pip install geopy

In [ ]:
from geopy.geocoders import Nominatim
import pandas as pd
from geopy.exc import GeocoderTimedOut

# Using the cities_df dataframe
cities_df = pd.DataFrame({

    'city': cities_df['city']  # Use all cities from cities_df
})

# Initialize geocoder
geolocator = Nominatim(user_agent="city_coordinates")

# Function to get coordinates for a city with retry mechanism
def get_coordinates_with_retry(city):
    max_retries = 3
    current_retry = 0

    while current_retry < max_retries:
        try:
            location = geolocator.geocode(city)
            if location:
                return location.latitude, location.longitude
            else:
                print(f"Coordinates not found for {city}")
                return None, None
        except GeocoderTimedOut:
            print(f"Timeout error for {city}. Retrying...")
            current_retry += 1

    print(f"Max retries exceeded for {city}. Skipping...")
    return None, None

# Get coordinates for each city with retry mechanism
city_coordinates = {city: get_coordinates_with_retry(city) for city in cities_df['city']}

# Create a DataFrame with city coordinates
coordinates_df = pd.DataFrame(city_coordinates.values(), index=city_coordinates.keys(), columns=['latitude', 'longitude'])

# Add the 'city' column to the DataFrame
# Another crucial steps because we need to connect it with the weatehr API
coordinates_df['city'] = coordinates_df.index

print(coordinates_df)

Coordinates not found for New Windor


                   latitude   longitude              city
Anchorage         61.216313 -149.894852         Anchorage
Atlanta           33.748992  -84.390264           Atlanta
Aguadilla         18.429792  -67.154220         Aguadilla
South Burlington  44.467163  -73.171567  South Burlington
Arlington         32.735582  -97.107119         Arlington
...                     ...         ...               ...
Gustavus          58.422399 -135.746228          Gustavus
Pago Pago        -14.275479 -170.704830         Pago Pago
Bishop            37.363679 -118.395240            Bishop
Loveland          40.385549 -105.044361          Loveland
Alpena            45.017618  -83.667002            Alpena

[403 rows x 3 columns]


In [ ]:
#checking for Null Values now

print(coordinates_df['latitude'].isnull().sum())
invalid_latitudes = coordinates_df[(coordinates_df['latitude'] < -90) | (coordinates_df['latitude'] > 90)]
print(invalid_latitudes)

1
Empty DataFrame
Columns: [latitude, longitude, city]
Index: []


In [ ]:
#I wasnt able to get the coordinates for New Windor
# But for now, I will remove New Windor from the data frame

coordinates_df.replace('', pd.NA, inplace=True)
# Removing any rows containing null values in any column
coordinates_df = coordinates_df.dropna()

# Reseting the index
coordinates_df.reset_index(drop=True, inplace=True)


In [ ]:
# Once agin checking for Null Values.
print(coordinates_df['latitude'].isnull().sum())
invalid_latitudes = coordinates_df[(coordinates_df['latitude'] < -90) | (coordinates_df['latitude'] > 90)]
print(invalid_latitudes)

# You will see 0 compared to the previous one

0
Empty DataFrame
Columns: [latitude, longitude, city]
Index: []


# Colleting Weather Data from Weather API
## Using Weather Api and using the cordinates that we had earlier to find the requirement that the proffessor gave us .
## such as temperature, humididty, precipitation whether rain or snow, wind speed

### Please note that the code was available from the weatehr Api website itself.

#### Please also note that there is two dataframe, I had to use two dataframes due to the fact that the weatehr API was prohibiting me to extract enourmosu amount of data fro free. thus the reason u will see 'combined_hourly_dataframe' and 'combined_hourly_df'.
#### Both these are concatonated to 'results_df' later.

#### Constructive feedback would be , i can make it more efficeient to make the code run in one block using algorithms but due to time limitations, I just crated two blocks of code with different time period.

In [ ]:
%%capture
!pip install openmeteo_requests
!pip install requests_cache
!pip install retry_requests

In [ ]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Initialize an empty list to store dataframes for each city
all_hourly_dataframes = []

# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"

# Iterate through each row in the DataFrame
for index, row in coordinates_df.iterrows():
    # Update the latitude and longitude in the params for each city
    params = {
        "latitude": row['latitude'],
        "longitude": row['longitude'],
        "start_date": "2023-05-28",
        "end_date": "2023-06-30",
        "hourly": ["temperature_2m", "relative_humidity_2m", "precipitation", "weather_code", "wind_speed_10m"]
    }

    # Fetch weather data for the current city
    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]  # Assuming one response per city

    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
    hourly_precipitation = hourly.Variables(2).ValuesAsNumpy()
    hourly_weather_code = hourly.Variables(3).ValuesAsNumpy()
    hourly_wind_speed_10m = hourly.Variables(4).ValuesAsNumpy()

    hourly_data = {
        "date": pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s"),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s"),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        )
    }

    # Add 'city' column to the hourly data
    hourly_data["city"] = row['city']

    hourly_data["temperature_2m"] = hourly_temperature_2m
    hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
    hourly_data["precipitation"] = hourly_precipitation
    hourly_data["weather_code"] = hourly_weather_code
    hourly_data["wind_speed_10m"] = hourly_wind_speed_10m

    # Create a new dataframe for the current city
    hourly_dataframe = pd.DataFrame(data=hourly_data)

    # Append the dataframe to the list
    all_hourly_dataframes.append(hourly_dataframe)

# Concatenate all dataframes into a single dataframe
combined_hourly_dataframe = pd.concat(all_hourly_dataframes, ignore_index=True)

combined_hourly_dataframe


                      date       city  temperature_2m  relative_humidity_2m  \
0      2023-05-28 00:00:00  Anchorage        9.164500             70.430420   
1      2023-05-28 01:00:00  Anchorage        9.014500             76.827232   
2      2023-05-28 02:00:00  Anchorage        9.514500             77.174721   
3      2023-05-28 03:00:00  Anchorage        9.464499             77.434357   
4      2023-05-28 04:00:00  Anchorage        8.614500             78.111382   
...                    ...        ...             ...                   ...   
328027 2023-06-30 19:00:00     Alpena       24.786999             71.733101   
328028 2023-06-30 20:00:00     Alpena       26.187000             62.990353   
328029 2023-06-30 21:00:00     Alpena       28.187000             49.379333   
328030 2023-06-30 22:00:00     Alpena       28.487000             42.553856   
328031 2023-06-30 23:00:00     Alpena       27.687000             42.609207   

        precipitation  weather_code  wind_speed_10m

In [ ]:
combined_hourly_dataframe

,date,city,temperature_2m,relative_humidity_2m,precipitation,weather_code,wind_speed_10m
0,2023-05-28 00:00:00,Anchorage,9.164500,70.430420,0.2,51.0,13.556282
1,2023-05-28 01:00:00,Anchorage,9.014500,76.827232,0.2,51.0,9.028754
2,2023-05-28 02:00:00,Anchorage,9.514500,77.174721,0.1,51.0,3.563818
3,2023-05-28 03:00:00,Anchorage,9.464499,77.434357,0.0,3.0,5.506941
4,2023-05-28 04:00:00,Anchorage,8.614500,78.111382,0.0,3.0,7.568566
...,...,...,...,...,...,...,...
328027,2023-06-30 19:00:00,Alpena,24.786999,71.733101,4.7,63.0,14.040000
328028,2023-06-30 20:00:00,Alpena,26.187000,62.990353,0.0,0.0,2.620839
328029,2023-06-30 21:00:00,Alpena,28.187000,49.379333,0.1,51.0,2.741678
328030,2023-06-30 22:00:00,Alpena,28.487000,42.553856,0.0,1.0,5.600286


In [ ]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Initialize an empty list to store dataframes for each city
all_hourly_dataframes = []

# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"

# Iterate through each row in the DataFrame
for index, row in coordinates_df.iterrows():
    # Update the latitude and longitude in the params for each city
    params = {
        "latitude": row['latitude'],
        "longitude": row['longitude'],
        "start_date": "2023-07-01",
        "end_date": "2023-08-28",
        "hourly": ["temperature_2m", "relative_humidity_2m", "precipitation", "weather_code", "wind_speed_10m"]
    }

    # Fetch weather data for the current city
    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]  # Assuming one response per city

    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
    hourly_precipitation = hourly.Variables(2).ValuesAsNumpy()
    hourly_weather_code = hourly.Variables(3).ValuesAsNumpy()
    hourly_wind_speed_10m = hourly.Variables(4).ValuesAsNumpy()

    hourly_data = {
        "date": pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s"),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s"),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        )
    }

    # Add 'city' column to the hourly data
    hourly_data["city"] = row['city']

    hourly_data["temperature_2m"] = hourly_temperature_2m
    hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
    hourly_data["precipitation"] = hourly_precipitation
    hourly_data["weather_code"] = hourly_weather_code
    hourly_data["wind_speed_10m"] = hourly_wind_speed_10m

    # Create a new dataframe for the current city
    hourly_df = pd.DataFrame(data=hourly_data)

    # Append the dataframe to the list
    all_hourly_dataframes.append(hourly_df)

# Concatenate all dataframes into a single dataframe
combined_hourly_df = pd.concat(all_hourly_dataframes, ignore_index=True)

combined_hourly_df



                      date       city  temperature_2m  relative_humidity_2m  \
0      2023-07-01 00:00:00  Anchorage       13.214499             77.233383   
1      2023-07-01 01:00:00  Anchorage       12.814500             76.135208   
2      2023-07-01 02:00:00  Anchorage       12.614500             77.138954   
3      2023-07-01 03:00:00  Anchorage       12.514500             79.502365   
4      2023-07-01 04:00:00  Anchorage       12.164500             83.010765   
...                    ...        ...             ...                   ...   
569227 2023-08-28 19:00:00     Alpena       23.536999             39.680405   
569228 2023-08-28 20:00:00     Alpena       24.286999             35.933640   
569229 2023-08-28 21:00:00     Alpena       24.536999             35.279778   
569230 2023-08-28 22:00:00     Alpena       22.737000             44.390545   
569231 2023-08-28 23:00:00     Alpena       19.737000             64.853485   

        precipitation  weather_code  wind_speed_10m

In [ ]:
combined_hourly_df

,date,city,temperature_2m,relative_humidity_2m,precipitation,weather_code,wind_speed_10m
0,2023-07-01 00:00:00,Anchorage,13.214499,77.233383,0.0,2.0,12.682018
1,2023-07-01 01:00:00,Anchorage,12.814500,76.135208,0.0,2.0,12.722830
2,2023-07-01 02:00:00,Anchorage,12.614500,77.138954,0.0,2.0,9.659814
3,2023-07-01 03:00:00,Anchorage,12.514500,79.502365,0.0,1.0,6.792466
4,2023-07-01 04:00:00,Anchorage,12.164500,83.010765,0.0,2.0,4.693826
...,...,...,...,...,...,...,...
569227,2023-08-28 19:00:00,Alpena,23.536999,39.680405,0.0,0.0,4.024922
569228,2023-08-28 20:00:00,Alpena,24.286999,35.933640,0.0,2.0,5.052841
569229,2023-08-28 21:00:00,Alpena,24.536999,35.279778,0.0,1.0,0.804985
569230,2023-08-28 22:00:00,Alpena,22.737000,44.390545,0.0,1.0,5.991594


In [ ]:
import pandas as pd

# Concatonating the dataframe as I mentioned earlier

# vertically (along rows)
results_df = pd.concat([combined_hourly_dataframe, combined_hourly_df], ignore_index=True)
results_df


,date,city,temperature_2m,relative_humidity_2m,precipitation,weather_code,wind_speed_10m
0,2023-05-28 00:00:00,Anchorage,9.164500,70.430420,0.2,51.0,13.556282
1,2023-05-28 01:00:00,Anchorage,9.014500,76.827232,0.2,51.0,9.028754
2,2023-05-28 02:00:00,Anchorage,9.514500,77.174721,0.1,51.0,3.563818
3,2023-05-28 03:00:00,Anchorage,9.464499,77.434357,0.0,3.0,5.506941
4,2023-05-28 04:00:00,Anchorage,8.614500,78.111382,0.0,3.0,7.568566
...,...,...,...,...,...,...,...
897259,2023-08-28 19:00:00,Alpena,23.536999,39.680405,0.0,0.0,4.024922
897260,2023-08-28 20:00:00,Alpena,24.286999,35.933640,0.0,2.0,5.052841
897261,2023-08-28 21:00:00,Alpena,24.536999,35.279778,0.0,1.0,0.804985
897262,2023-08-28 22:00:00,Alpena,22.737000,44.390545,0.0,1.0,5.991594


In [ ]:
# Creating a datetime
results_df['datetime'] = pd.to_datetime(results_df['date'])
results_df

NameError: ignored

In [ ]:
# Redefine the desired order of columns
desired_column_order = ['city', 'datetime', 'temperature_2m', 'relative_humidity_2m', 'precipitation', 'weather_code', 'wind_speed_10m']

results_df = results_df[desired_column_order]
results_df


,city,datetime,temperature_2m,relative_humidity_2m,precipitation,weather_code,wind_speed_10m
0,Anchorage,2023-05-28 00:00:00,9.164500,70.430420,0.2,51.0,13.556282
1,Anchorage,2023-05-28 01:00:00,9.014500,76.827232,0.2,51.0,9.028754
2,Anchorage,2023-05-28 02:00:00,9.514500,77.174721,0.1,51.0,3.563818
3,Anchorage,2023-05-28 03:00:00,9.464499,77.434357,0.0,3.0,5.506941
4,Anchorage,2023-05-28 04:00:00,8.614500,78.111382,0.0,3.0,7.568566
...,...,...,...,...,...,...,...
897259,Alpena,2023-08-28 19:00:00,23.536999,39.680405,0.0,0.0,4.024922
897260,Alpena,2023-08-28 20:00:00,24.286999,35.933640,0.0,2.0,5.052841
897261,Alpena,2023-08-28 21:00:00,24.536999,35.279778,0.0,1.0,0.804985
897262,Alpena,2023-08-28 22:00:00,22.737000,44.390545,0.0,1.0,5.991594


In [ ]:
# Seeing the tpye of dataframe of all the columns for the TSA data
column_types = combined_df.dtypes.to_dict()

for col, col_type in column_types.items():
    print(f"{col}: {col_type}")

city: object
datetime: datetime64[ns]
airportcode: object
airportname: object
state: object
checkpoint: object
total: object


In [ ]:
# Seeing the tpye of dataframe of all the columns for the Weather Data
column_types = results_df.dtypes.to_dict()

for col, col_type in column_types.items():
    print(f"{col}: {col_type}")

city: object
datetime: datetime64[ns]
temperature_2m: float32
relative_humidity_2m: float32
precipitation: float32
weather_code: float32
wind_speed_10m: float32


#### After i checked if the type of columns I am merginsg matches.
### I use the merge function to connect the two data frame of TSA data and Weatehr data
### SI connect them using the 'datatime'parameter and also connecting them to the city.

In [ ]:
import pandas as pd

# Merging the data
merged_df = pd.merge(combined_df, results_df, on=['city', 'datetime'], how='left')
merged_df



,city,datetime,airportcode,airportname,state,checkpoint,total,temperature_2m,relative_humidity_2m,precipitation,weather_code,wind_speed_10m
0,Anchorage,2023-05-28 00:00:00,ANC,Ted Stevens Anchorage International,AK,South Checkpoint,113,9.164500,70.430420,0.2,51.0,13.556282
1,Atlanta,2023-05-28 00:00:00,ATL,Hartsfield Atlanta International,GA,Main Checkpoint,142,18.411501,50.383205,0.0,1.0,20.898611
2,Aguadilla,2023-05-28 00:00:00,BQN,Rafael Hernandez,PR,Rafael Hernandez Air,268,25.007000,95.328812,0.0,1.0,3.219938
3,South Burlington,2023-05-28 00:00:00,BTV,Burlington International,VT,Main Checkpoint,0,21.143999,54.437481,0.0,0.0,6.989935
4,Arlington,2023-05-28 00:00:00,DCA,Washington Reagan National,VA,Concourse A,1,27.882999,43.359516,0.0,1.0,8.161764
...,...,...,...,...,...,...,...,...,...,...,...,...
697142,San Jose,2023-08-26 23:00:00,SJC,Norman Y Mineta San Jose International,CA,Terminal A,0,25.612001,42.873352,0.0,0.0,20.799152
697143,San Juan,2023-08-26 23:00:00,SJU,Luis Munoz Marin International,PR,Checkpoint BC,238,25.870001,89.821571,0.1,51.0,6.151683
697144,Salt Lake City,2023-08-26 23:00:00,SLC,Salt Lake City International,UT,Main,84,32.712997,21.179127,0.0,1.0,7.412853
697145,Sacramento,2023-08-26 23:00:00,SMF,Sacramento International,CA,Central B,184,35.348003,16.765631,0.0,0.0,12.979984


# Saving the cleaned output to azure blob
#### Last but not the least, we save it to azure blob for back up and for further analysis such as Azure Synapse/ Redshift/ Big Query

In [ ]:
# Save the combined front-filled DataFrame to a CSV file
merged_df.to_csv("A-M_Merged.csv", index=False)

# Azure Storage Account information
account_name = "your account"
account_key = "your account key"
container_name = "jsoncme"
blob_name = 'A-M_Merged.csv'
local_file_path = 'your path'

# BlobServiceClient
blob_service_client = BlobServiceClient(account_url=f"https://{account_name}.blob.core.windows.net", credential=account_key)

# reference to the container
container_client = blob_service_client.get_container_client(container_name)

# Uploading the file to Azure Blob Storage
with open(local_file_path, "rb") as data:
    container_client.upload_blob(name=blob_name, data=data)

print(f"File '{blob_name}' uploaded to Azure Blob Storage successfully. /n Congratulations")


File 'A-M_Merged.csv' uploaded to Azure Blob Storage successfully. /n Congratulations
